In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "https://nhacnheo.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
songs = []


In [46]:
import requests
from bs4 import BeautifulSoup

# Function to fetch HTML content
def fetch_html(url):
    response = requests.get(url)
    return response.content

# Function to extract data from the main table
def extract_data_from_table(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    rows = soup.find('tbody').find_all('tr')
    results = []

    for row in rows:
        # Get the 2nd <td> link
        second_td = row.find_all('td')[1]
        second_link = second_td.find('a')['href']

        # Get the 4th <td> link
        fourth_td = row.find_all('td')[3]
        fourth_link = fourth_td.find('a')['href']

        # Fetch and process the images from the second link
        images = fetch_images_from_link(second_link)

        # Fetch and process details from the fourth link
        title, author, tags, lyrics = fetch_details_from_link(fourth_link)

        results.append({
            'second_link_images': images,
            'fourth_link_title': title,
            'fourth_link_author': author,
            'fourth_link_tags': tags,
            'fourth_link_lyrics': lyrics
        })

    return results

# Function to fetch images from a link
def fetch_images_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    image_tags = soup.select('div.ct-box img')
    images = [img['src'] for img in image_tags]
    return images

# Function to fetch title, author, tags, and lyrics from a link
def fetch_details_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract title
    title = soup.find('h1', class_='h3 d-inline-block').text.strip()

    # Extract author
    author_section = soup.find(text=lambda x: "Sáng tác" in x)
    author = ""
    if author_section:
        author = author_section.split("Sáng tác:")[1].split('|')[0].strip()

    # Extract tags
    tags_section = soup.find('i', class_='fas fa-book')
    tags = ""
    if tags_section:
        tags = tags_section.find_next('a').text.strip()

    # Extract lyrics
    lyrics_div = soup.find('div', id='lyric')
    if lyrics_div:
        # for span in lyrics_div.find_all('span'):
        #     span.extract()
        # Delete all the <span></span> content inside lyrics_div
        [span.extract() for span in lyrics_div.find_all('span')]
        
        lyrics = lyrics_div.text.strip()

    return title, author, tags, lyrics

# Main URL
url = "https://hopamviet.vn/sheet/index/"

# Get the results
data = extract_data_from_table(url)

# Output the results (you can save this to a file or process it further)
for item in data:
    print("Title:", item['fourth_link_title'])
    print("Author:", item['fourth_link_author'])
    print("Tags:", item['fourth_link_tags'])
    print("Lyrics:", item['fourth_link_lyrics'])
    print("Images:", item['second_link_images'])
    print("-" * 40)


C:\Users\hanhn\AppData\Local\Temp\ipykernel_3800\3016634967.py:60: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  author_section = soup.find(text=lambda x: "Sáng tác" in x)


NameError: name 'images' is not defined

In [3]:
import requests
from bs4 import BeautifulSoup
import re

# Function to fetch HTML content
def fetch_html(url):
    response = requests.get(url)
    return response.content

# Function to extract data from a specific <tr>
def extract_data_from_row(url, row_index=0):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all <tr> elements in the <tbody>
    rows = soup.find('tbody').find_all('tr')
    
    if not rows or row_index >= len(rows):
        print(f"No <tr> found at index {row_index}.")
        return
    
    # Get the specified <tr>
    row = rows[row_index]
    
    # Get the 2nd <td> link
    second_td = row.find_all('td')[1]
    second_link = second_td.find('a')['href']
    print("Second Link:", second_link)

    # Get the 4th <td> link
    fourth_td = row.find_all('td')[3]
    fourth_link = fourth_td.find('a')['href']
    print("Fourth Link:", fourth_link)

    # Fetch and process the images from the second link
    # images = fetch_images_from_link(second_link)
    files = fetch_files_from_link(second_link)
    # print("Images:", images)
    print("Files:", files)

    # Fetch and process details from the fourth link
    title, author, tags, lyrics = fetch_details_from_link(fourth_link)

    print("Title:", title)
    print("Author:", author)
    print("Tags:", tags)
    print("Lyrics:\n", lyrics)

# Function to fetch images and other files from a link
def fetch_files_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Fetch image sources from <img> tags
    image_tags = soup.select('div.ct-box img')
    images = [img['src'] for img in image_tags]
    
    # Fetch PDF or other file sources from <iframe> tags
    iframe_tags = soup.select('div.ct-box iframe')
    files = []
    for iframe in iframe_tags:
        src = iframe.get('src')
        if src and "google.com/viewer?url=" in src:
            # Extract the actual file URL from the Google Viewer link
            file_url = src.split("url=")[1].split("&")[0]
            files.append(file_url)
    
    # Combine images and files into a single list
    files.extend(images)
    
    return files
# Function to fetch title, author, tags, and lyrics from a link
def fetch_details_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract title
    title = soup.find('h1', class_='h3 d-inline-block').text.strip()

    # Extract author names
    author_section = soup.find('i', title="Sáng tác")
    author = ""
    if author_section:
        # Get the entire author text including the links
        author_text_node = author_section.find_next_sibling(text=True)
        if author_text_node:
            # Find all <a> tags before encountering the next "|" symbol
            author_links = []
            current = author_text_node.find_next_sibling('a')
            while current and current.name == 'a':
                author_links.append(current.text.strip())
                current = current.find_next_sibling()
                if isinstance(current, str) and '|' in current:
                    break
            author = ' & '.join(author_links)

    # Extract tags
    tags_section = soup.find('i', class_='fas fa-book')
    tags = ""
    if tags_section:
        tags = tags_section.find_next('a').text.strip()

    # Extract lyrics
    lyrics_div = soup.find('div', id='lyric')
    lyrics = lyrics_div.text.strip()
    
    # Remove all chords in the format [Am], [Dm], etc. using regex
    lyrics = re.sub(r'\[.*?\] ', '', lyrics)
    # FInd if in the lyrics there are any chords in the format [Am], [Dm], etc.
    # chords = re.findall(r'\[.*?\]', lyrics)
    # Remove all of them
    lyrics = re.sub(r'\[.*?\]', '', lyrics)    

    return title, author, tags, lyrics

# Main URL
# url = "https://hopamviet.vn/sheet/index/"
url = "https://hopamviet.vn/sheet/index/700"

# Test with a specific <tr> for debugging
# For example, test with the second <tr> (index 1)
extract_data_from_row(url, row_index=12)


Second Link: https://hopamviet.vn/sheet/song/careless-whisper/W8IUIZAF.html
Fourth Link: https://hopamviet.vn/chord/song/careless-whisper/W8IUIZAF.html
Files: ['https://hopamviet.vn/assets/images/sheets/Careless-band-tone-nampdf.pdf']
Title: Careless whisper
Author: George Michael
Tags: Nhạc Trữ tình
Lyrics:
 Intro: x 2

I feel so un- sure as I take your hand and lead you to the dance floor 
As the music dies, something in your eyes 
Calls to mind a silver screen and all its sad good- bye 

I'm never gonna dance again, these guilty feet have got no rhythm 
Though it's easy to pretend I know you're not a fool 
I Should've known better than to cheat a friend 
And waste a chance that I've been given 
So I'm never gonna dance again the way I danced with you oh oh

Time can never mend the careless whisper of a good friend
To the heart and mind, Ignorance is kind
There's no comfort in the truth, pain is the hole you'll find

I'm never gonna dance again, these guilty feet have got no rhythm 


C:\Users\hanhn\AppData\Local\Temp\ipykernel_10496\504977393.py:85: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  author_text_node = author_section.find_next_sibling(text=True)


In [7]:
# from airflow import DAG
# from airflow.operators.python_operator import PythonOperator
# from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import re
# import firebase_admin
# from firebase_admin import credentials, firestore, storage
import tempfile
import os
from urllib.parse import urlparse, unquote

# # Firebase setup
# cred = credentials.Certificate("/opt/airflow/dags/firebase-service-account.json")
# firebase_admin.initialize_app(cred, {
#     'storageBucket': 'braille-music-library.appspot.com'
# })
# db = firestore.client()
# bucket = storage.bucket()

def fetch_html(url):
    response = requests.get(url)
    return response.content

def extract_data_from_row(url, row_index=0):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    rows = soup.find('tbody').find_all('tr')
    
    if not rows or row_index >= len(rows):
        print(f"No <tr> found at index {row_index}.")
        return None
    
    row = rows[row_index]
    
    second_td = row.find_all('td')[1]
    second_link = second_td.find('a')['href']
    
    fourth_td = row.find_all('td')[3]
    fourth_link = fourth_td.find('a')['href']

    files = fetch_files_from_link(second_link)
    title, author, tags, lyrics = fetch_details_from_link(fourth_link)
    
    song = {
        "title": title,
        "author": author,
        "tags": tags,
        "lyrics": lyrics,
        "files": files,
        "second_link": second_link,
        "fourth_link": fourth_link,
    }

    return song

def fetch_files_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    image_tags = soup.select('div.ct-box img')
    images = [img['src'] for img in image_tags]
    
    iframe_tags = soup.select('div.ct-box iframe')
    files = []
    for iframe in iframe_tags:
        src = iframe.get('src')
        if src and "google.com/viewer?url=" in src:
            file_url = src.split("url=")[1].split("&")[0]
            files.append(file_url)
    
    files.extend(images)
    
    return files

def fetch_details_from_link(url):
    html_content = fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')
    
    title = soup.find('h1', class_='h3 d-inline-block').text.strip()

    author_section = soup.find('i', title="Sáng tác")
    author = ""
    if author_section:
        author_text_node = author_section.find_next_sibling(text=True)
        if author_text_node:
            author_links = []
            current = author_text_node.find_next_sibling('a')
            while current and current.name == 'a':
                author_links.append(current.text.strip())
                current = current.find_next_sibling()
                if isinstance(current, str) and '|' in current:
                    break
            author = ' & '.join(author_links)

    tags_section = soup.find('i', class_='fas fa-book')
    tags = ""
    if tags_section:
        tags = tags_section.find_next('a').text.strip()

    lyrics_div = soup.find('div', id='lyric')
    lyrics = lyrics_div.text.strip()
    
    lyrics = re.sub(r'\[.*?\] ', '', lyrics)
    lyrics = re.sub(r'\[.*?\]', '', lyrics)    

    return title, author, tags, lyrics

def upload_to_database(song):
    query = db.collection('hopamviet').where('title', '==', song['title']).limit(1)
    existing_docs = list(query.stream())

    if len(existing_docs) == 0:
        file_name = f"{song['title'].replace(' ', '_')}.pdf"
        song_data = {
            "title": song['title'],
            "author": song['author'],
            "tags": song['tags'],
            "lyrics": song['lyrics'],
            "origin_url": song.get('files', []),
            "braille_url": None,
            "timestamp": firestore.SERVER_TIMESTAMP
        }
        db.collection('hopamviet').add(song_data)
    else:
        existing_doc_ref = existing_docs[0].reference
        existing_doc_ref.update({
            "author": song['author'],
            "tags": song['tags'],
            "lyrics": song['lyrics'],
            "origin_url": song.get('files', []),
            "braille_url": None,
            "timestamp": firestore.SERVER_TIMESTAMP
        })

def upload_file_to_firebase(file_url, title):
    # Determine the file extension from the URL
    parsed_url = urlparse(file_url)
    file_name = os.path.basename(parsed_url.path)
    file_name = unquote(file_name)  # Decode any URL-encoded characters
    file_extension = os.path.splitext(file_name)[1]

    # Ensure the title is URL-safe and append the file extension
    safe_title = title.replace(' ', '_')
    file_name = f"{safe_title}{file_extension}"

    blob = bucket.blob(f"origin/hopamviet/{file_name}")
    if not blob.exists():
        response = requests.get(file_url)
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(response.content)
            temp_file.seek(0)
            blob.upload_from_file(temp_file)
        return blob.public_url
    else:
        return blob.public_url

def crawl_pages(start_url, end_url, step=3):
    base_url = start_url.rsplit('/', 1)[0]
    start_index = int(start_url.rsplit('/', 1)[1])
    end_index = int(end_url.rsplit('/', 1)[1])

    songs = []
    for i in range(start_index, end_index + 1, step):
        url = f"{base_url}/{i}"
        try:
            html_content = fetch_html(url)
            soup = BeautifulSoup(html_content, 'html.parser')
            rows = soup.find('tbody').find_all('tr')
            max_row_index = min(99, len(rows) - 1)  # Crawl up to row 99 or the max row index
            for row_index in range(max_row_index + 1):
                song = extract_data_from_row(url, row_index=row_index)
                if song:
                    songs.append(song)
        except Exception as e:
            print(f"Error processing {url}: {e}")
    return songs




In [10]:
crawl_pages("https://hopamviet.vn/sheet/index/0", "https://hopamviet.vn/sheet/index/400", step=100)

C:\Users\hanhn\AppData\Local\Temp\ipykernel_10496\1513088167.py:86: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  author_text_node = author_section.find_next_sibling(text=True)


[{'title': '100 phần trăm',
  'author': 'Ngọc Sơn & Tuấn Hải',
  'tags': 'Nhạc Trữ tình',
  'lyrics': 'Một trăm em ơi chiều nay một trăm phần trăm\nMột trăm em ơi chiều nay một trăm phần trăm\nNgười yêu anh ơi giờ đây lại cắm trại rồi\nNào đâu nào biết tâm tư người lính\nLòng anh nao nao mỗi khi ta hẹn nhau với em tâm tình\n\nXin em nhớ cho rằng chuyện tình biết truớc từ đầu\nDẫu lính hay đa tình nhưng\nMãi mãi vẫn yêu chỉ yêu một người\nMột người mà thôi và yêu trọn đời\nMột giây về phép anh xin dành cho em đó\n\nMột trăm em ơi chiều nay một trăm phần trăm\nMột trăm em ơi chiều nay một trăm phần trăm\nNày em yêu ơi vì anh là lính chung tình\nĐừng ghen và nhớ xin em đừng hờn dỗi\nChiều nay không em thấy yêu em nhiều hơn cớ sao em buồn',
  'files': ['https://hopamviet.vn/assets/images/sheets/100_bia_(1).jpg',
   'https://hopamviet.vn/assets/images/sheets/100_Tram_01.jpg',
   'https://hopamviet.vn/assets/images/sheets/100_Tram_02.jpg'],
  'second_link': 'https://hopamviet.vn/sheet/song/1